### Welcome !! 
to this *hands-on* Blockchain development workshop

This workshop will focus on the Immutability/Tamper-proof property of blockchain

#### Immutablity
Immutable means that something is not changed over time or cannot be changed.

#### Hashes
Almost all blockchain system uses hashes (especially cryptographic hashes) to make things immutable

#### Blocks
Blocks organize **transaction** in a certain data structure and add other metadata to it and seal it with a hash. The hash of a block is calculated from the data *(contents)*. 

#### Blockchain
Blockchains are an **append only** data structure to links blocks. When a new block is being created the **hash** of the last block in the blockchain is also added as a metadata into this block. As blocks contain the hash of the previous block as metadata in this block, a **block-*chain*** is created.


### How is immutablitiy achieved ?
1. Block hash *(hash is calculated out of the contents of the block)*
2. When a block hash fails, the whole chain fails *(every block has the hash of previous block)*
3. Recreating the whole chain is infeasible
    1. Proof-of-work
    2. Private signatures

### Pre-requisites
Following are the things needed 
1. Any programming language
2. Hashing library

This workshop is created with Python, but feel free to implement this in any language that you are familiar with.

### Some Python Basics
* List
* Slicing
* Iterator
* Dict

In [2]:
# This is a list
mylist = [1,1,2,3,5,8]

In [5]:
len(mylist)

7

In [4]:
mylist.append(13)
mylist

[1, 1, 2, 3, 5, 8, 13]

List can be sliced as follows

In [6]:
# List slicing
mylist[1:3]

[1, 2]

In [7]:
mylist[:3]


[1, 1, 2]

In [8]:
mylist[-1]


13

In [9]:
mylist[-2:]

[8, 13]

### Iterators

Any python type that impletements `__iter__()` method becomes an iterator. It can then be used in `for in [Object]` loops


In [10]:
for item in mylist:
    print(item)

1
1
2
3
5
8
13


### Dict 
Dictionaries are another builtin datatype like associative arrays in other languages

In [12]:
person = { 'name': "Satheesh", 'country': "India", 'age': 32}
person

{'age': 32, 'country': 'India', 'name': 'Satheesh'}

### Crypto Hash
1. it is __deterministic__ so the same message always results in the same hash
2. it is quick to compute the hash value for any given message
3. it is __infeasible__ to generate a message from its hash value except by trying all possible messages
4. a small change to a message should change the hash value so extensively that the new hash value appears uncorrelated with the old hash value
5. it is __infeasible__ to find two different messages with the same hash value

In [15]:
import hashlib
hashlib.sha256("Blockchain is simple").hexdigest()

'bcfea943fb6d2944749b028b857dcafe0879d4ccd395d95c95e84124e384078f'

In [16]:
hashlib.sha256("Blockchain is Simple").hexdigest()

'a6cd67f548ed9f28c8288a90daf4fadd389a325b0f654489adaab4107667fbdf'

### jsonpickle
jsonpickle is a library to serialize and deserialize python types to json

In [8]:
import jsonpickle
jsonpickle.encode(("test", 1, 3, "four"))

'{"py/tuple": ["test", 1, 3, "four"]}'

In [9]:
import datetime
import hashlib
from time import time
import jsonpickle

### Transaction
Transaction type stores information about a transaction, this is a simplified version of a transaction. In reality it will have addition things like 
1. Input address
2. Output address
3. Signature for this transaction
4. Units

Based on these information the transaction id is generated

In [14]:
class Transaction:
    """
    Transaction stores the following information
    1. Sender
    2. Receiver
    3. Amount
    4. Timestamp
    
    TODO: Add a generated transaction ID based on some hashing technique 
    """
    def __init__(self, sender, receiver, amount):
        self.sender = sender
        self.receiver = receiver
        self.amount = amount
        self.timestamp = time()
            
    def __repr__(self):
        return 'Transaction : {} sent {} units to {}'.format(
            self.sender, self.amount, self.receiver
        )

In [15]:
a = Transaction("Satheesh", "Chaitra", 10)
b = Transaction("Chaitra", "Nala", 5)

In [16]:
print(a)
print(b)

Transaction : Satheesh sent 10 units to Chaitra
Transaction : Chaitra sent 5 units to Nala


#### Hash

```
hash=f(data)
```

In [3]:
def hash_message(data):
    return hashlib.sha256(data).hexdigest()

### Block
Block are a way of organizing the transactions that also stores some metadata.



In [10]:
class Block:
    def __init__(self, blockchain = None):
        self.transactions = []
        self.prev_hash = None
        self.height = None
        if blockchain is not None:
            self.prev_hash = blockchain[-1].hash
            self.height = len(blockchain) + 1 
        else:
            self.height = 1
        self.hash = None
        self.timestamp = time()
        #self.payload_hash = self._hash_payload()
        self.transaction_count = 0;
        
    
    def _hash_payload(self):
        return self._hash_transactions()

    def _hash_transactions(self):
        curr_hash = ""
        for tx in self.transactions:
            txn_rep = curr_hash.join(jsonpickle.encode(tx))
            curr_hash = hashlib.sha256(txn_rep).hexdigest()
        return curr_hash
    
    def add_transaction(self, transaction):
        self.transactions.append(transaction)
        self.transaction_count = len(self.transactions)

    def _hash_block(self):
        self.payload_hash = self._hash_payload()
        blockheader_data = {
            'payload_hash' : self.payload_hash,
            'timestamp' : self.timestamp,
            'prev_hash' : self.prev_hash,
            'total_transactions' : self.transaction_count
        }
        block_rep = jsonpickle.encode(blockheader_data)
        return hashlib.sha256(block_rep).hexdigest()

    def finalize(self):
        self.hash = self._hash_block()
    
    def validate(self):
        return self.hash == self._hash_block()

In [33]:
block = Block()

In [34]:
block.add_transaction(a)
block.add_transaction(b)

In [35]:
block.transaction_count

2

In [19]:
# setting jsonpickle options
# 1. Sort the object by keys so we get consistent hash
# 2. indent=4 for readablity
jsonpickle.set_encoder_options("json",sort_keys=True, indent=4)

In [36]:
block.finalize()

In [37]:
block.validate()

True

In [22]:
c = Transaction("Nala", "Simba", 1)

In [23]:
block.add_transaction(c)

In [24]:
block.validate()

False

In [38]:
myblockchain = []

In [39]:
myblockchain.append(block)

### Validation
Lets validate the entire blockchain by looping through and calling validate method on each block

In [40]:
def validatechain(blockchain):
    print(len(blockchain))
    for block in blockchain:
        if block.validate():
            print("Block #{} is valid".format(block.height))
            if block.height != 1:
                if block.prev_hash != blockchain[block.height-2].hash:
                    print("Prev hash {} != Prev block hash {}".format(block.prev_hash,
                                                                      blockchain[block.height-2].hash))
                    return False
                print("Block #{} Prev hash {} == Prev block hash {}".format(block.height, block.prev_hash,
                                                                            blockchain[block.height-2].hash))
        else:
            print ("Block {} is invalid".format(block.height))
            return False
    else:
        return True

In [41]:
validatechain(myblockchain)

1
Block #1 is valid


True

In [42]:
a = Transaction("Vintoh", "Jeeva", 10)

In [43]:
newblock = Block(myblockchain)
newblock.add_transaction(a)
newblock.finalize()

In [44]:
myblockchain.append(newblock)

In [45]:
validatechain(myblockchain)

2
Block #1 is valid
Block #2 is valid
Block #2 Prev hash c1daa2ee513ea8ae61d8cca6d2b4288ea94997133a24b927d3e32dcd6fd79aa7 == Prev block hash c1daa2ee513ea8ae61d8cca6d2b4288ea94997133a24b927d3e32dcd6fd79aa7


True

### Functions to store and load the blockchain from FS

In [46]:
def savechain(blockchain, datafile):
    with open(datafile,"wb") as bf:
        bf.write(jsonpickle.encode(blockchain))

In [47]:
def loadchain(datafile):
    with open(datafile) as bf:
        return jsonpickle.loads(bf.read())

In [48]:
savechain(myblockchain, "blockchain.json")

### Inspect
Now lets inspect the saved blockchain datastructure in the json format

In [49]:
savedblockchain = loadchain("blockchain.json")

In [50]:
validatechain(savedblockchain)

2
Block #1 is valid
Block #2 is valid
Block #2 Prev hash c1daa2ee513ea8ae61d8cca6d2b4288ea94997133a24b927d3e32dcd6fd79aa7 == Prev block hash c1daa2ee513ea8ae61d8cca6d2b4288ea94997133a24b927d3e32dcd6fd79aa7


True